In [1]:
import wordsearch as ws
from IPython.display import HTML, display, FileLink

%load_ext autoreload
%autoreload 2

In [2]:
wordlist = ['camel','tunnel','squirrel','travel','towel','tinsel','bagel','hazel','vowel','jewel','cry','fly','dry','try','reply','sly','shy','terrify','sky','multiply']

In [3]:
ws.make_wordsearch_pack(wordlist, "11th December 2020: Spelling Wordsearch ")

In [8]:
display(FileLink("Easy.html")) 

C:\Users\boylesa\Desktop\Wordsearch\Easy.html

In [9]:
display(FileLink("Medium.html"))

C:\Users\boylesa\Desktop\Wordsearch\Medium.html

In [10]:
display(FileLink("Hard.html"))

C:\Users\boylesa\Desktop\Wordsearch\Hard.html

In [7]:
display(FileLink("answers.txt"))

C:\Users\boylesa\Desktop\Wordsearch\answers.txt